In [1]:
from vaeNet import *
import torch
f = diffAttn(embed_dim=12,input_chn=10,layer_index=1)

t = torch.ones((64,12,10))
ff = f(t)
print(ff.shape)

torch.Size([64, 12, 512])
torch.Size([64, 12, 512])
F.shape
torch.Size([12, 1024])
torch.Size([64, 12, 512])
torch.Size([64, 12, 512])
torch.Size([64, 12, 512])
v.shape: torch.Size([64, 512, 12])
torch.Size([64, 512, 512])
torch.Size([64, 512, 12])
torch.Size([64, 512, 12])
